In [1]:
import numpy as np

In [2]:
import tensorflow as tf

In [52]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Concatenate,Conv2DTranspose,Input

In [53]:
from tensorflow.keras.layers import Activation

In [54]:
def encoder_block(inputs,num_filters):
    p=Conv2D(num_filters,3,padding="valid")(inputs)
    p=Activation("relu")(p)
    q=Conv2D(num_filters,3,padding="valid")(p)
    q=Activation("relu")(q)
    r=MaxPool2D(pool_size=(2,2),strides=2)(q)
    return r

In [55]:
def bottle_neck_block(inputs,num_filters):
    p=Conv2D(num_filters,3,padding="valid")(inputs)
    p=Activation("relu")(p)
    q=Conv2D(num_filters,3,padding="valid")(p)
    q=Activation("relu")(q)
    return q

In [56]:
def decoder_block(inputs,skip_connections,num_filters):
    p=Conv2DTranspose(num_filters,(2,2),strides=2,padding="valid")(inputs)
    skip_connections=tf.image.resize(skip_connections,size=(p.shape[1],p.shape[2]))
    combined=Concatenate()([skip_connections+p])
    q=Conv2D(num_filters,3,padding="valid")(combined)
    q=Activation("relu")(q)
    r=Conv2D(num_filters,3,padding="valid")(q)
    r=Activation("relu")(r)
    return r

In [57]:
def output_block(inputs,outputs):
    p=Conv2D(outputs,1,padding="valid",activation="sigmoid")(inputs)
    return p

In [58]:
inputs=Input(shape=(572,572,3))

p=encoder_block(inputs,64)
q=encoder_block(p,128)
r=encoder_block(q,256)
s=encoder_block(r,512)

a=bottle_neck_block(s,1024)

c=decoder_block(a,s,512)
d=decoder_block(c,r,256)
e=decoder_block(d,q,128)
f=decoder_block(e,p,64)

g=output_block(f,2)

In [59]:
from tensorflow.keras import Model

In [60]:
model=Model(inputs,g,name="U-NET")

In [61]:
model.summary()

Model: "U-NET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 572, 572, 3)]        0         []                            
                                                                                                  
 conv2d_109 (Conv2D)         (None, 570, 570, 64)         1792      ['input_7[0][0]']             
                                                                                                  
 activation_86 (Activation)  (None, 570, 570, 64)         0         ['conv2d_109[0][0]']          
                                                                                                  
 conv2d_110 (Conv2D)         (None, 568, 568, 64)         36928     ['activation_86[0][0]']       
                                                                                              

 tf.__operators__.add_22 (T  (None, 104, 104, 256)        0         ['tf.image.resize_22[0][0]',  
 FOpLambda)                                                          'conv2d_transpose_22[0][0]'] 
                                                                                                  
 concatenate_22 (Concatenat  (None, 104, 104, 256)        0         ['tf.__operators__.add_22[0][0
 e)                                                                 ]']                           
                                                                                                  
 conv2d_121 (Conv2D)         (None, 102, 102, 256)        590080    ['concatenate_22[0][0]']      
                                                                                                  
 activation_98 (Activation)  (None, 102, 102, 256)        0         ['conv2d_121[0][0]']          
                                                                                                  
 conv2d_12

In [62]:
x=np.random.random((572,572,3))